# CaLPA Scratch Folder

This is a scratch notebook used to test the code and functionality of the AI California Legislative Policy Analysis (CALPA) system. It is not intended for production use and may contain incomplete or experimental code. The purpose of this notebook is to facilitate the development and testing of the CALPA system, including its data processing, analysis, and visualization components. The notebook may include code snippets, comments, and notes related to the development process. Please refer to the official documentation and user guides for the CALPA system for more information on its usage and features.

In [ ]:
# %reset

## Initialization

In [11]:
# Import required libraries
import os
import time
from datetime import date
from datetime import datetime
import json
import mimetypes
import glob
import base64
import zipfile
import io
import dotenv
import requests
import pandas as pd
import feedparser
import webbrowser

In [2]:
# Load the Calpa module located in the scripts/python/calpa directory
from calpa import Calpa, LegiScan

# Load environment variables from .env file
dotenv.load_dotenv(os.path.join(os.getcwd(), '.env'))

# Instantiate the LegiScan and Calpa classes
calpa = Calpa()
legiscan = LegiScan()

# Create project metadata for the AI project
prjMetadata = calpa.projectMetadata("AI", "0")

# Create the project directories dictionary
prjDirs = calpa.projectDirectories(os.getcwd())

Project Global Settings:
- Name: California Legislative Policy Analysis
- Title: AI Legislative Policy Analysis
- Version: 1.0
- Author: Dr. Kostas Alexandridis, GISP
Data Dates
- Start Date: 2010-12-02
- End Date: 2025-04-20
- Periods: 2009-2010, 2011-2012, 2013-2014, 2015-2016, 2017-2018, 2019-2020, 2021-2022, 2023-2024, 2025-2026
Directory Global Settings:

General:
- Project: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA
- Admin: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\admin
- Metadata: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\metadata
- Analysis: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\analysis
Scripts:
- Python Calpa Module: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\calpa
- Markdown Scripts: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\markdown
- RIS Scripts: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\ris
Data:
- Main Data: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\data
- Documents: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\data\docs
- Le

In [3]:
# Load the getBillCodebook pickle file from the data/lookup directory
getBillCodebook = pd.read_pickle(os.path.join(prjDirs["pathDataLookup"], "getBillCodebook.pkl"))

# Load the getRollCallCodebook pickle file from the data/lookup directory
getRollCallCodebook = pd.read_pickle(os.path.join(prjDirs["pathDataLookup"], "getRollCallCodebook.pkl"))

# Load the getBillTextCodebook pickle file from the data/lookup directory
getBillTextCodebook = pd.read_pickle(os.path.join(prjDirs["pathDataLookup"], "getBillTextCodebook.pkl"))

# Load the getAmendmentCodebook pickle file from the data/lookup directory
getAmendmentCodebook = pd.read_pickle(os.path.join(prjDirs["pathDataLookup"], "getAmendmentCodebook.pkl"))

# Load the getSupplementCodebook pickle file from the data/lookup directory
getSupplementCodebook = pd.read_pickle(os.path.join(prjDirs["pathDataLookup"], "getSupplementCodebook.pkl"))

# Load the getPersonCodebook pickle file from the data/lookup directory
getPersonCodebook = pd.read_pickle(os.path.join(prjDirs["pathDataLookup"], "getPersonCodebook.pkl"))

# Load the getSessionListCodebook pickle file from the data/lookup directory
getSessionListCodebook = pd.read_pickle(os.path.join(prjDirs["pathDataLookup"], "getSessionListCodebook.pkl"))

In [6]:
# Obtain the stored sessions list from JSON dictionary on disk (data/lookup directory)
sessionListStored = legiscan.getStoredData(type="session")

# Obtain the stored session People list from JSON dictionary on disk (data/lookup directory)
sessionPeopleStored = legiscan.getStoredData(type="people")

# Obtain the stored dataset list from JSON dictionary on disk (data/lookup directory)
datasetListStored = legiscan.getStoredData(type="dataset")

# Get the stored raw master list from JSON dictionary on disk (data/lookup directory)
masterListRawStored = legiscan.getStoredData(type="master", raw=True)
# Get the stored master list from JSON dictionary on disk (data/lookup directory)
masterListStored = legiscan.getStoredData(type="master", raw=False)

# Get the AI monitoring list from disk (data/lookup directory)
aiBillListStored = legiscan.getStoredData(type="bills", project="AI")

# Get the AI full list of bills from dism (data/legis/json directory)
aiBills = legiscan.getStoredData(type="data", project="AI")

In [15]:
webbrowser.open(aiBills["2013-2014"]["AB1465"]["url"], new=2, autoraise=True)

True

In [16]:
webbrowser.open(aiBills["2013-2014"]["AB1465"]["state_link"], new=2, autoraise=True)

True

In [19]:
aiBills["2013-2014"]["AB1465"]["progress"][-1]["date"]

'2014-06-15'

In [22]:
aiBills["2013-2014"]["AB1465"]["history"][-1]

{'date': '2014-06-26',
 'action': 'Ordered to inactive file at the request of Senator Corbett.',
 'chamber': 'S',
 'chamber_id': 20,
 'importance': 0}

In [26]:
# for each of the sponsors in aiBills["2013-2014"]["AB1465"]["sponsors"] find all the sponsors that have sponsor_type_id == 1
for sponsor in aiBills["2013-2014"]["AB1465"]["sponsors"]:
    if sponsor["sponsor_type_id"] == 2:
        print(sponsor["name"])

In [30]:
[f"{sponsor['name']} ({sponsor['party']}, {sponsor['district'].replace('-','')})" for sponsor in aiBills["2013-2014"]["AB1465"]["sponsors"] if sponsor["sponsor_type_id"] == 1]

['Nancy Skinner (D, SD009)',
 'Richard Bloom (D, HD050)',
 'Nora Campos (D, HD027)',
 'Wesley Chesbro (D, HD002)',
 'Matt Dababneh (D, HD045)',
 'Tom Daly (D, HD069)',
 'Roger Dickinson (D, HD007)',
 'Richard Gordon (D, HD024)',
 'Reginald Jones-Sawyer (D, HD059)',
 'Kevin Mullin (D, HD022)',
 'Al Muratsuchi (D, HD066)',
 'Adrin Nazarian (D, HD046)',
 'Freddie Rodriguez (D, HD052)',
 'Mark Stone (D, HD029)',
 'Philip Ting (D, HD019)',
 'Shirley Weber (D, HD079)']

In [35]:
legiscan.sponsorType[0]

'Sponsor'

In [86]:
legiscan.summarizeBillSponsors(aiBills["2013-2014"]["SB836"], output="dict")

{'Primary Sponsor': ['Ellen Corbett (D, SD10)'],
 'Co-Sponsor': ['Joel Anderson (R, SD38)',
  'Jim Beall (D, SD15)',
  'Marty Block (D, SD39)',
  'Anthony Cannella (R, SD12)',
  'Lou Correa (D, SD34)',
  'Noreen Evans (D, SD02)',
  'Cathleen Galgiani (D, SD05)',
  'Loni Hancock (D, SD09)',
  'Jerry Hill (D, SD13)',
  'Ricardo Lara (D, SD33)',
  'Ted Lieu (D, SD28)',
  'Carol Liu (D, SD25)',
  'Jim Nielsen (R, SD04)',
  'Richard Roth (D, SD31)',
  'Lois Wolk (D, SD03)'],
 'Joint Sponsor': ['Nancy Skinner (D, SD09)', 'Marie Waldron (R, AD75)']}

In [87]:
legiscan.summarizeBillSponsors(aiBills["2013-2014"]["SB836"], output="md")

- Primary Sponsor(s): [Ellen Corbett (D, SD10)](https://ballotpedia.org/Ellen_Corbett)
- Co-Sponsor(s): [Joel Anderson (R, SD38)](https://ballotpedia.org/Joel_Anderson_(California)), [Jim Beall (D, SD15)](https://ballotpedia.org/James_Beall_Jr.), [Marty Block (D, SD39)](https://ballotpedia.org/Martin_Block), [Anthony Cannella (R, SD12)](https://ballotpedia.org/Anthony_Cannella), [Lou Correa (D, SD34)](https://ballotpedia.org/Lou_Correa), [Noreen Evans (D, SD02)](https://ballotpedia.org/Noreen_Evans), [Cathleen Galgiani (D, SD05)](https://ballotpedia.org/Cathleen_Galgiani), [Loni Hancock (D, SD09)](https://ballotpedia.org/Loni_Hancock), [Jerry Hill (D, SD13)](https://ballotpedia.org/Gerald_Hill), [Ricardo Lara (D, SD33)](https://ballotpedia.org/Ricardo_Lara), [Ted Lieu (D, SD28)](https://ballotpedia.org/Ted_Lieu), [Carol Liu (D, SD25)](https://ballotpedia.org/Carol_Liu), [Jim Nielsen (R, SD04)](https://ballotpedia.org/Jim_Nielsen), [Richard Roth (D, SD31)](https://ballotpedia.org/Richar

{'Primary Sponsor': '[Ellen Corbett (D, SD10)](https://ballotpedia.org/Ellen_Corbett)',
 'Co-Sponsor': '[Joel Anderson (R, SD38)](https://ballotpedia.org/Joel_Anderson_(California)), [Jim Beall (D, SD15)](https://ballotpedia.org/James_Beall_Jr.), [Marty Block (D, SD39)](https://ballotpedia.org/Martin_Block), [Anthony Cannella (R, SD12)](https://ballotpedia.org/Anthony_Cannella), [Lou Correa (D, SD34)](https://ballotpedia.org/Lou_Correa), [Noreen Evans (D, SD02)](https://ballotpedia.org/Noreen_Evans), [Cathleen Galgiani (D, SD05)](https://ballotpedia.org/Cathleen_Galgiani), [Loni Hancock (D, SD09)](https://ballotpedia.org/Loni_Hancock), [Jerry Hill (D, SD13)](https://ballotpedia.org/Gerald_Hill), [Ricardo Lara (D, SD33)](https://ballotpedia.org/Ricardo_Lara), [Ted Lieu (D, SD28)](https://ballotpedia.org/Ted_Lieu), [Carol Liu (D, SD25)](https://ballotpedia.org/Carol_Liu), [Jim Nielsen (R, SD04)](https://ballotpedia.org/Jim_Nielsen), [Richard Roth (D, SD31)](https://ballotpedia.org/Richar

- Primary Sponsor(s): [Ellen Corbett (D, SD10)](https://ballotpedia.org/Ellen_Corbett)
- Co-Sponsor(s): [Joel Anderson (R, SD38)](https://ballotpedia.org/Joel_Anderson_(California)), [Jim Beall (D, SD15)](https://ballotpedia.org/James_Beall_Jr.), [Marty Block (D, SD39)](https://ballotpedia.org/Martin_Block), [Anthony Cannella (R, SD12)](https://ballotpedia.org/Anthony_Cannella), [Lou Correa (D, SD34)](https://ballotpedia.org/Lou_Correa), [Noreen Evans (D, SD02)](https://ballotpedia.org/Noreen_Evans), [Cathleen Galgiani (D, SD05)](https://ballotpedia.org/Cathleen_Galgiani), [Loni Hancock (D, SD09)](https://ballotpedia.org/Loni_Hancock), [Jerry Hill (D, SD13)](https://ballotpedia.org/Gerald_Hill), [Ricardo Lara (D, SD33)](https://ballotpedia.org/Ricardo_Lara), [Ted Lieu (D, SD28)](https://ballotpedia.org/Ted_Lieu), [Carol Liu (D, SD25)](https://ballotpedia.org/Carol_Liu), [Jim Nielsen (R, SD04)](https://ballotpedia.org/Jim_Nielsen), [Richard Roth (D, SD31)](https://ballotpedia.org/Richard_Roth), [Lois Wolk (D, SD03)](https://ballotpedia.org/Lois_Wolk)
- Joint Sponsor(s): [Nancy Skinner (D, SD09)](https://ballotpedia.org/Nancy_Skinner_(California)), [Marie Waldron (R, AD75)](https://ballotpedia.org/Marie_Waldron)

## Bill Sponsors
- [Ellen Corbett (D, SD10)](https://ballotpedia.org/Ellen_Corbett)

In [36]:
for sponsor in aiBills["2013-2014"]["AB1465"]["sponsors"]:
    print(legiscan.sponsorType[sponsor["sponsor_type_id"]])

Primary Sponsor
Primary Sponsor
Primary Sponsor
Primary Sponsor
Primary Sponsor
Primary Sponsor
Primary Sponsor
Primary Sponsor
Primary Sponsor
Primary Sponsor
Primary Sponsor
Primary Sponsor
Primary Sponsor
Primary Sponsor
Primary Sponsor
Primary Sponsor


In [48]:
print("[Nancy Skinner (D, SD09)](https://ballotpedia.org/Nancy_Skinner_(California))")

[Nancy Skinner (D, SD09)](https://ballotpedia.org/Nancy_Skinner_(California))


In [ ]:
aiBills["2013-2014"]["AB1465"]["sponsors"]

[{'people_id': 1561,
  'person_hash': 'aeniw3oh',
  'party_id': '1',
  'state_id': 5,
  'party': 'D',
  'role_id': 1,
  'role': 'Rep',
  'name': 'Nancy Skinner',
  'first_name': 'Nancy',
  'middle_name': 'C.',
  'last_name': 'Skinner',
  'suffix': '',
  'nickname': '',
  'district': 'SD-009',
  'ftm_eid': 81851,
  'votesmart_id': 104350,
  'opensecrets_id': '',
  'knowwho_pid': 267715,
  'ballotpedia': 'Nancy_Skinner_(California)',
  'bioguide_id': '',
  'sponsor_type_id': 1,
  'sponsor_order': 1,
  'committee_sponsor': 0,
  'committee_id': 0,
  'state_federal': 0},
 {'people_id': 14092,
  'person_hash': 'vls7twp3',
  'party_id': '1',
  'state_id': 5,
  'party': 'D',
  'role_id': 1,
  'role': 'Rep',
  'name': 'Richard Bloom',
  'first_name': 'Richard',
  'middle_name': '',
  'last_name': 'Bloom',
  'suffix': '',
  'nickname': '',
  'district': 'HD-050',
  'ftm_eid': 11022796,
  'votesmart_id': 70734,
  'opensecrets_id': '',
  'knowwho_pid': 390005,
  'ballotpedia': 'Richard_Bloom',
  '

In [10]:
aiBills["2013-2014"]["AB1465"]

{'bill_id': 581806,
 'change_hash': '20bb8b3a704cdb43730e1ecee46b680d',
 'session_id': 993,
 'session': {'session_id': 993,
  'state_id': 5,
  'year_start': 2013,
  'year_end': 2014,
  'prefile': 0,
  'sine_die': 1,
  'prior': 1,
  'special': 0,
  'session_tag': 'Regular Session',
  'session_title': '2013-2014 Regular Session',
  'session_name': '2013-2014 Session'},
 'url': 'https://legiscan.com/CA/bill/AB1465/2013',
 'state_link': 'http://www.leginfo.ca.gov/cgi-bin/postquery?bill_number=ab_1465&sess=1314&house=A',
 'completed': 0,
 'status': 2,
 'status_date': '2014-05-23',
 'progress': [{'date': '2014-01-09', 'event': 1},
  {'date': '2014-05-23', 'event': 2},
  {'date': '2014-05-23', 'event': 9},
  {'date': '2014-06-05', 'event': 9},
  {'date': '2014-06-12', 'event': 10},
  {'date': '2014-06-12', 'event': 9},
  {'date': '2014-06-15', 'event': 10}],
 'state': 'CA',
 'state_id': 5,
 'bill_number': 'AB1465',
 'bill_type': 'B',
 'bill_type_id': '1',
 'body': 'A',
 'body_id': 19,
 'curre